<a id='fi-basics-top'></a>
# Fixed Income Analysis<br><span style="color:orange"> BQuant Basics seminar series </span>
### <a href="https://platform.cinchcast.com/ses/r1A0CrXOw_c0bIQtko2uSw~~">Click here to watch the webinar</a>

Before digesting this notebook, we recommend that you are comfortable with some basic BQL concepts, such as where to find the existing documentation, navigating the BQNT environment, filtering, and aggregation.

This notebook serves as an introduction to our data sets, and key data transformations available for fixed income/credit analysis in BQuant.

<a href="https://blinks.bloomberg.com/screens/BQIQ%20L%235288759">Click here for all upcoming BQuant webinars </a>

<a id='s0'></a>

Content:<br>
- [1  - Intro](#s1)<a href='#s1'></a>  
- [2  - Credit Universes](#s1.5)<a href='#s1'></a>  
- [3  - Chains](#s2.0)<a href='#s2.0'></a>
- [4  - Index Analysis](#s3.0)<a href='#s3.0'></a>
- [5  - Momentum Analysis](#s4.0)<a href='#s4.0'></a>
- [6  - Issuer Analysis](#s5.0)<a href='#s5.0'></a>
- [7  - Summary](#s6.0)<a href='#s6.0'></a>


In [ ]:
import bql
import numpy as np

In [ ]:
bq = bql.Service()

<a id='s1'></a>
### 1. Credit Data
- Clean, standardized data.
- Easier to reference and manipulate.

[&uarr; Return to Top](#fi-basics-top)

<a id='s1.5'></a>
### 2. Universes

There are several universes we can use when working with BQL in BQNT for credit data:

* <b>bq.univ.bondsuniv()</b>: This represents all bonds which you can filter, best used with the "active" parameter to exclude all matured bonds. 

* <b>bq.univ.bonds()</b>: to pull all bonds for another universe. ie:
 *  bq.univ.bonds("VOD LN Equity")
 *  bq.univ.bonds(bq.univ.members("SPX Index")) 
 
* <b>bq.univ.loans()</b>: to pull loans
* <b>bq.univ.debt()</b>: to pull bonds, loans, preferreds


[&uarr; Return to Top](#fi-basics-top)

<a id='s2.0'></a>

### 2. Chains

- Use chains to access bonds for single / multiple issuers and select the corporate structure.
- Use BQL to screen and aggregate standardized fixed income data.
- Generate your custom peer yield curve on the fly.

##### Q: What is the yield curve for fixed coupon USD bonds from MSFT, IBM and ORCL?

In [ ]:
#Define our universe of bonds issued by issuers. 
#The issuedby parameters allow you to select the level of the corporate structure bonds come from.
bonds = bq.univ.bonds(['MSFT US Equity', 'IBM US Equity','ORCL US Equity'], 
                      issuedby = 'Credit_Family')
#Define our screening criteria of fixed coupon USD bonds
coupon_typ = bq.data.cpn_typ() == 'Fixed'
currency = bq.data.crncy() == 'USD'
rating = bq.data.rtg_fitch() > "AA" # note for ratings we can use these operators. 
criteria = bq.func.and_(coupon_typ, currency)
criteria = bq.func.and_(criteria, rating)
#Apply filter
filtered_bond_chains = bq.univ.filter(bonds, criteria)
#Calculate and retrieve median yield by year of maturity
yld = bq.data.yield_()
mat = bq.data.maturity().year()
yld_by_mat = yld.group(mat).median()
#Request data, handle response
req = bql.Request(filtered_bond_chains, {'Custom Peer Yield Curve':yld_by_mat})
res = bq.execute(req)
data = res[0].df()
data

[&uarr; Return to Top](#fi-basics-top)

<a id='s3.0'></a>

### 3. Benchmark Analysis
- Breakdown an index by custom categories to an instant overview.
- Make weighted average calculations to accurately reflect index composition.

##### Q: What is the total amount outstanding by S&P credit rating for bonds in LF98TRUU Index with maturity after 2029?

In [ ]:
#Define our universe as the members of an index
index = bq.univ.members('LF98TRUU Index')
#Define our maturity filter and apply
criteria_mat = bq.data.maturity() >= '2028-12-31'
filtered_index = bq.univ.filter(index, criteria_mat)
#Calculate and retrieve the total amount outstandng (scaled in 10^6) by S&P rating
amt_out = bq.data.amt_outstanding(CURRENCY = 'USD')
rating = bq.data.credit_rating(CREDIT_RATING_SOURCE = 'SANDP').let("rating")
amt_out_by_rating = amt_out.group(rating).sum() / 10 ** 6
#Request datam handle response
req = bql.Request(filtered_index, {'Amt Out by Rating':amt_out_by_rating})
res = bq.execute(req)
data = res[0].df()
data

##### Q: What is the weighted average spread by sector in LBUSTRUU?

In [ ]:
#Define universe of index members
members = bq.univ.members('LBUSTRUU Index')
#Screen and apply to clean our datasets for NaN
sprd = bq.data.spread(SPREAD_TYPE = 'OAS')
# please note the proper way to establish if a variable is nan
# is np.isnan(), but in the bql object model you can use != np.nan 
cleaned_index = bq.univ.filter(members,  sprd != np.nan )
#Calculate and retrieve the weighted average spread by sector
#Note weights is an associated column for id, and we can refer to it
#for calculations as follows:
weights = bq.data.ID()['WEIGHTS']
sector = bq.data.classification_name()
weighted_avg = sprd.group(sector).wavg(weights.group(sector))
#Request data and handle the response
req = bql.Request(cleaned_index, {'Weighted Avg Spread by Sector':weighted_avg})
res = bq.execute(req)
data = res[0].df()
data

[&uarr; Return to Top](#fi-basics-top)

<a id='s4.0'></a>

### 4. Momentum Analysis
- Screen according to the movement of bond spreads, prices, yields etc.

##### Q: What were the top 10 bonds in LF98TRUU with the greatest I-Spread widening last month?

In [ ]:
#Define our universe of index members
index = bq.univ.members('LF98TRUU Index')
#Define spread change over the last month, note we can use datetime
#or date in the string format as well
sprd = bq.data.spread(SPREAD_TYPE='I', dates=bq.func.range('-1m','0d'), fill='prev')
sprd_chg = sprd.net_chg()
#Rank our monthly spread change values from largest to smallest
sprd_chg_rank = sprd_chg.grouprank(order = 'desc')
#Filter by top 10 rank 
criteria = sprd_chg_rank <= 10
filtered_index = bq.univ.filter(index, criteria)
#Request data and handle responsem 
req = bql.Request(filtered_index, {'Top 10 Bonds to Watch':sprd_chg.round('3').groupsort()})
res = bq.execute(req)
data = res[0].df()
data

[&uarr; Return to Top](#fi-basics-top)

<a id='s5.0'></a>

### 5. Issuer Analysis
- Cross reference fixed income data with equity data.
- Have all your analysis in one single place.

##### Q: Who has issued USD green bonds since 2019?

In [ ]:
# Count the number of USD green bonds issued by issuer since 2019 
all_bonds = bq.univ.bondsuniv('All')
# Apply filters
criteria_issue_dt = bq.data.issue_dt() >= '2019-01-01' 
criteria_green = bq.data.green_bond_loan_indicator() == 'True'
criteria_crncy = bq.data.crncy() == 'USD'
# Combine our filter criteria
green_criteria = bq.func.and_(criteria_crncy, bq.func.and_(criteria_green, criteria_issue_dt))
green_filter = bq.univ.filter(all_bonds, green_criteria)
# Count the bonds in our filtered universe by issuer
security = bq.data.id()
name = bq.data.name()
# The value function allows you to put a query within a query, with a different universe. 
# mapby resolves the relationship between the universe in the outer query 
# and the universe in the inner query. In this case mapby="lineage" resolves the 
# relationship between bond and issuer.
# basically, we are taking a group of bonds, getting their issuer's equity id, 
# getting the name of the equity, and mapping it to each bond's id. 
issuer_name = name.value(bq.univ.issuerof(), mapby = 'lineage')
# number of green bonds issued by each issuer. 
count_by_issuer = security.group(issuer_name).count()
# request data and handle response
req = bql.Request(green_filter, {'Green Bonds by Issuer':count_by_issuer.groupsort()})
res = bq.execute(req)
data = res[0].df().head(30)
data

In [ ]:
port = bq.univ.members('U12533411-2', type='port')
sec = bq.data.id()

req = bql.Request(port, sec)
res = bq.execute(req)
data = res[0].df()
data

[&uarr; Return to Top](#fi-basics-top)

<a id='s6.0'></a>

### 6. Summary

- BQL lets you access bond data from chains, indices, portfolios and entire universes.
- You can easily build your analysis such as:
    - Custom Peer Curve.
    - Benchmark Rating breakdown.
    - Momentum Watchlists.
    - Green Bond Issuance.
- Get in touch for more on powerful data-efficient analysis!
- BQuant Basics Series: [Introduction to BQuant](https://platform.cinchcast.com/ses/jXAdkrDNj-lPJiwE5vbzIA~~) | [Credit/Fixed Income](https://platform.cinchcast.com/ses/r1A0CrXOw_c0bIQtko2uSw~~) | [Widgets](https://platform.cinchcast.com/ses/SsWJrxoSVMbm1RrNEAcGvw~~) | [Funds](https://platform.cinchcast.com/ses/nf84iygPJ7urt0lyUYIOCQ~~) | [Equity](https://onlinexperiences.com/scripts/Server.nxp?LASCmd=AI:4;F:QS!10100&ShowUUID=AB1C7DAA-0558-4D4D-B8DA-159BB315B169) | [Charts](http://bloomberglp.com/BqntBasicsCharts) |

[&uarr; Return to Top](#fi-basics-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
     <a href="../../0 Welcome.ipynb">&larr; Back to the Home</a>&emsp;&emsp;
    <a href="#fi-basics-top">&uarr; Back to Top </a>&emsp;&emsp;
    <a href="../Learn/1.1. FI Basics - Workbook.ipynb">Next Topic: 1.1 FI Basics &rarr;</a>
    <br>

</p>